<a href="https://colab.research.google.com/github/anqi-guo/anomaly_detection_isolation_forest/blob/main/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 读取数据

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import os

path = '/content/drive/My Drive/work/在线监测/data'
df = pd.read_csv(f'{path}/t_zxyc_fs_ssj_processed.csv')
df['监测时间'] = pd.to_datetime(df['监测时间'], dayfirst=True)
df.head()

Mounted at /content/drive


,排口ID,污染物ID,监测时间,污染物浓度值,污染物排放量,排放量,流量,污染物浓度上限值,是否生产,生产状态_流量中位数
0,32028120181214320281010,w01001,2021-04-13 08:00:00,7.219,0.000,52.932,14.703,9.0,1,7.615
1,32028120181214320281010,w21003,2021-04-13 08:00:00,0.072,0.004,52.932,14.703,20.0,1,7.615
2,430420171008ROOT002,w21011,2021-04-13 08:00:00,0.088,0.096,1101.175,305.882,0.5,1,205.248
3,430420171008ROOT002,w01018,2021-04-13 08:00:00,35.822,39.447,1101.175,305.882,50.0,1,205.248
4,430420171008ROOT002,w21001,2021-04-13 08:00:00,8.748,9.633,1101.175,305.882,15.0,1,205.248


# 删掉无效数据

In [ ]:
cnt = df.groupby(['排口ID','污染物ID']).size()

max_time = df.groupby(['排口ID','污染物ID'])['监测时间'].max()
valid_data = max_time[max_time==df['监测时间'].max()]

valid_data = valid_data.to_frame()
valid_data.reset_index(inplace=True)

valid_df = df.merge(valid_data[['排口ID','污染物ID']], on=['排口ID','污染物ID'], how='right')
valid_df.head()

,排口ID,污染物ID,监测时间,污染物浓度值,污染物排放量,排放量,流量,污染物浓度上限值,是否生产,生产状态_流量中位数
0,32028120180404320281002,w01001,2021-04-13 06:00:00,7.012,0.0,295.695,82.138,9.0,1,62.942
1,32028120180404320281002,w01001,2021-04-13 14:00:00,7.012,0.0,297.558,82.655,9.0,1,62.942
2,32028120180404320281002,w01001,2021-04-13 22:00:00,7.021,0.0,286.817,79.671,9.0,1,62.942
3,32028120180404320281002,w01001,2021-04-13 16:00:00,7.018,0.0,267.592,74.331,9.0,1,62.942
4,32028120180404320281002,w01001,2021-04-13 00:00:00,7.031,0.0,280.755,77.988,9.0,1,62.942


# 加入企业和地理信息

In [ ]:
static_data = pd.read_csv(f'{path}/静态数据.csv')

valid_df = valid_df.merge(static_data[['企业名称','城市名称','区县名称','排口ID']], on='排口ID', how='left')
valid_df.head()

,排口ID,污染物ID,监测时间,污染物浓度值,污染物排放量,排放量,流量,污染物浓度上限值,是否生产,生产状态_流量中位数,企业名称,城市名称,区县名称
0,32028120180404320281002,w01001,2021-04-13 06:00:00,7.012,0.0,295.695,82.138,9.0,1,62.942,亚同环保水处理江阴有限公司,周庄镇,江阴市
1,32028120180404320281002,w01001,2021-04-13 14:00:00,7.012,0.0,297.558,82.655,9.0,1,62.942,亚同环保水处理江阴有限公司,周庄镇,江阴市
2,32028120180404320281002,w01001,2021-04-13 22:00:00,7.021,0.0,286.817,79.671,9.0,1,62.942,亚同环保水处理江阴有限公司,周庄镇,江阴市
3,32028120180404320281002,w01001,2021-04-13 16:00:00,7.018,0.0,267.592,74.331,9.0,1,62.942,亚同环保水处理江阴有限公司,周庄镇,江阴市
4,32028120180404320281002,w01001,2021-04-13 00:00:00,7.031,0.0,280.755,77.988,9.0,1,62.942,亚同环保水处理江阴有限公司,周庄镇,江阴市


In [ ]:
valid_df.info(memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8104103 entries, 0 to 8104102
Data columns (total 13 columns):
 #   Column      Dtype         
---  ------      -----         
 0   排口ID        object        
 1   污染物ID       object        
 2   监测时间        datetime64[ns]
 3   污染物浓度值      float64       
 4   污染物排放量      float64       
 5   排放量         float64       
 6   流量          float64       
 7   污染物浓度上限值    float64       
 8   是否生产        int64         
 9   生产状态_流量中位数  float64       
 10  企业名称        object        
 11  城市名称        object        
 12  区县名称        object        
dtypes: datetime64[ns](1), float64(6), int64(1), object(5)
memory usage: 865.6+ MB


# 更改数据格式

In [ ]:
for c in valid_df.select_dtypes(include=['object','int64']).columns:
  valid_df[c] = valid_df[c].astype('category')

for c in valid_df.select_dtypes(include=['float64']).columns:
  valid_df[c] = valid_df[c].astype('float32')

valid_df.info(memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8104103 entries, 0 to 8104102
Data columns (total 13 columns):
 #   Column      Dtype         
---  ------      -----         
 0   排口ID        category      
 1   污染物ID       category      
 2   监测时间        datetime64[ns]
 3   污染物浓度值      float32       
 4   污染物排放量      float32       
 5   排放量         float32       
 6   流量          float32       
 7   污染物浓度上限值    float32       
 8   是否生产        category      
 9   生产状态_流量中位数  float32       
 10  企业名称        category      
 11  城市名称        category      
 12  区县名称        category      
dtypes: category(6), datetime64[ns](1), float32(6)
memory usage: 371.0 MB


# 增加基于业务规则判定的异常数据列

In [ ]:
from tqdm.auto import tqdm
tqdm.pandas()

abnormal_data = pd.read_csv(f'{path}/FS_Y.csv')

for c in ['XXKSSJ','JSJZSJ']:
  abnormal_data[c] = pd.to_datetime(abnormal_data[c], dayfirst=True)

abnormal_data['监测时间'] = abnormal_data.progress_apply(lambda row: pd.date_range(start=row['XXKSSJ'], end=row['JSJZSJ'], freq='H'), axis=1)
abnormal_data = abnormal_data.explode('监测时间')
abnormal_data.rename(columns={'PKBH':'排口ID', 'WRWBH':'污染物ID'}, inplace=True)
abnormal_data['是否异常'] = 1

for c in ['排口ID','污染物ID']:
  abnormal_data[c] = abnormal_data[c].astype('category')

del abnormal_data['XXKSSJ']
del abnormal_data['JSJZSJ']

valid_df = valid_df.merge(abnormal_data, on=['监测时间','污染物ID','排口ID'], how='left')

del abnormal_data

import gc
gc.collect()

valid_df.head()

  0%|          | 0/630119 [00:00<?, ?it/s]

,排口ID,污染物ID,监测时间,污染物浓度值,污染物排放量,排放量,流量,污染物浓度上限值,是否生产,生产状态_流量中位数,企业名称,城市名称,区县名称,是否异常
0,32028120180404320281002,w01001,2021-04-13 06:00:00,7.012,0.0,295.695007,82.138000,9.0,1,62.942001,亚同环保水处理江阴有限公司,周庄镇,江阴市,NaN
1,32028120180404320281002,w01001,2021-04-13 14:00:00,7.012,0.0,297.558014,82.654999,9.0,1,62.942001,亚同环保水处理江阴有限公司,周庄镇,江阴市,NaN
2,32028120180404320281002,w01001,2021-04-13 22:00:00,7.021,0.0,286.816986,79.670998,9.0,1,62.942001,亚同环保水处理江阴有限公司,周庄镇,江阴市,NaN
3,32028120180404320281002,w01001,2021-04-13 16:00:00,7.018,0.0,267.592010,74.331001,9.0,1,62.942001,亚同环保水处理江阴有限公司,周庄镇,江阴市,NaN
4,32028120180404320281002,w01001,2021-04-13 00:00:00,7.031,0.0,280.755005,77.987999,9.0,1,62.942001,亚同环保水处理江阴有限公司,周庄镇,江阴市,NaN


In [ ]:
valid_df['是否异常'].fillna(0, inplace=True)
valid_df['是否异常'].value_counts(normalize=True)

0.0    0.805458
1.0    0.194542
Name: 是否异常, dtype: float64

In [ ]:
valid_df.to_csv(f'{path}/unprocessed_data.csv', index=False)